# Ether data formatting and analysis

In [14]:
import pm4py
import csv
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime
import os
import pickle

In [3]:
# ingest data on internal transactions, normal transactions and gas cost
path_internal = "<path-to-internal-transaction-data>"
path_normal = "<path-to-normal-transaction-data>"
path_internal_gas = "<path-to-internal-gas-data>"

df_internal = pd.read_csv(path_internal, delimiter=",", dtype={"blockNumber": int, "timeStamp": "string", "hash":"string", "from": "string", "to":"string", "value": "string", "gas":"string", "gasUsed":"string", "isError":int, "input":"string"})
df_normal = pd.read_csv(path_normal, delimiter=",", dtype={"blockNumber": int, "timeStamp": "string", "hash":"string", "from": "string", "to":"string", "value": "string", "gas":"string", "gasUsed":"string", "isError":int, "input":"string"})
df_internal_gas = pd.read_csv(path_internal_gas, delimiter=",", dtype={"hash":"string", "gasUsedNormal":int, "gasUsedInternal":int, "gasPriceNormal":int})

In [4]:
# calculate count of users
user_addresses_count_internal = df_internal['to'].value_counts()
user_addresses_count_normal = df_normal['from'].value_counts()

In [5]:
df_internal_full = pd.DataFrame.merge(df_internal, df_internal_gas, on='hash')
df_internal_full = df_internal_full.replace(pd.NA, "0")

# convert strings to numbers
df_internal_full["value_ETH"] = df_internal_full['value'].str[:-15]
df_internal["value_ETH"] = df_internal['value'].str[:-15]
df_normal["value_ETH"] = df_normal['value'].str[:-15]
# convert strings to numbers
df_internal_full["value_ETH"] = pd.to_numeric(df_internal_full.value_ETH)*0.001
df_internal["value_ETH"] = pd.to_numeric(df_internal.value_ETH)*0.001
df_normal["value_ETH"] = pd.to_numeric(df_normal.value_ETH)*0.001
df_internal_full["gasUsedNormal"] = pd.to_numeric(df_internal_full.gasUsedNormal)
df_normal["gasUsed"] = pd.to_numeric(df_normal.gasUsed)
df_internal_full["gasPriceNormal"] = pd.to_numeric(df_internal_full.gasPriceNormal)
df_normal["gasPrice"] = pd.to_numeric(df_normal.gasPrice)
# convert timestamp
df_internal_full['timeStamp_UTC'] = df_internal_full['timeStamp'].apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d %H:%M:%S:%ms'))
df_internal['timeStamp_UTC'] = df_internal['timeStamp'].apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d %H:%M:%S:%ms'))
df_normal['timeStamp_UTC'] = df_normal['timeStamp'].apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d %H:%M:%S:%ms'))

In [6]:
# convert Wei to Ether
df_normal["transaction_fee_ETH"] = df_normal.gasUsed*df_normal.gasPrice/10**18
df_internal_full["transaction_fee_ETH"] = df_internal_full.gasUsedNormal*df_internal_full.gasPriceNormal/10**18

In [7]:
# create a balance sheet and sum-up income, spendings, and transaction fees for user accounts
df_balance = pd.DataFrame(columns=["address", "sum_income", "sum_spendings"])
income = pd.DataFrame(df_internal.groupby(["to"])["value_ETH"].sum())
income.rename(columns={"value_ETH":"income_ETH"}, inplace=True)
spendings = pd.DataFrame(df_normal.groupby(["from"])["value_ETH"].sum())
spendings.rename(columns={"value_ETH":"spendings_ETH"}, inplace=True)
fees = pd.DataFrame(df_normal.groupby(["from"])["transaction_fee_ETH"].sum())
fees.rename(columns={"transaction_fee_ETH":"fees_ETH"}, inplace=True)

# concatenate the values in one table
balance = pd.concat([spendings, income, fees], axis=1)
balance.fillna(0, inplace=True)

# calculate total spendings
balance["total_ETH"]=balance.income_ETH-balance.spendings_ETH-balance.fees_ETH

# reorder columns
balance = balance.reindex(columns=["income_ETH", "spendings_ETH", "fees_ETH", "total_ETH"])
balance.reset_index(inplace=True)
balance.rename(columns={"index":"address"}, inplace=True)

In [8]:
# calculate stats

overall = balance.spendings_ETH.sum()-balance.income_ETH.sum()
fees_total = balance.fees_ETH.sum()
income_total = balance.income_ETH.sum()

print(f"Overall balance (spendings-income): {overall}")
print(f"Total fees: {fees_total}")
print(f"Total income: {income_total}")

Overall balance (spendings-income): 40161.67899999977
Total fees: 39650.25488903612
Total income: 728720.6500000001


In [16]:
# save for later use

curr_dir = os.getcwd()
dir_path = os.path.dirname(curr_dir)
file = "balance"
path = os.path.join(dir_path, "resources", file + ".pkl")
pickle.dump(balance, open(path, 'wb'))